In [3]:
# Create the Spark Session
from pyspark.sql import SparkSession
from pyspark.sql import types as T
from pyspark.sql import window as W
from pyspark.sql import functions as F
from dataset.fakedata import create_fakeuser

spark = (
    SparkSession 
    .builder 
    .appName("06_write_to_multiple_databases") 
    .config("spark.streaming.stopGracefullyOnShutdown", True) 
    .config('spark.jars.packages', 'org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.0')
    .config("spark.driver.extraClassPath", "./jdbc/mysql-connector-j-8.4.0.jar:./jdbc/postgresql-42.7.3.jar") \
    .config("spark.sql.shuffle.partitions", 8)
    .master("local[*]") 
    .getOrCreate()
)

#### 1. mysql, postgresql connection test

In [4]:
sql = "SELECT * FROM pg_settings"

df_pg = spark.read.format("jdbc") \
                    .option("url", f"jdbc:postgresql://dockercompose-postgres-1:5432/postgres") \
                    .option("driver", "org.postgresql.Driver") \
                    .option("query", sql) \
                    .option("user", "yein") \
                    .option("password", "941021") \
                    .load()

df_pg.show(3)

+--------------------+--------------------+----+--------------------+--------------------+----------+---------+-------+-------+-------+-------+--------+--------+---------+----------+----------+---------------+
|                name|             setting|unit|            category|          short_desc|extra_desc|  context|vartype| source|min_val|max_val|enumvals|boot_val|reset_val|sourcefile|sourceline|pending_restart|
+--------------------+--------------------+----+--------------------+--------------------+----------+---------+-------+-------+-------+-------+--------+--------+---------+----------+----------+---------------+
|allow_in_place_ta...|                 off|null|   Developer Options|Allows tablespace...|      null|superuser|   bool|default|   null|   null|    null|     off|      off|      null|      null|          false|
|allow_system_tabl...|                 off|null|   Developer Options|Allows modificati...|      null|superuser|   bool|default|   null|   null|    null|     off

In [5]:
sql = "select * from fakeuser"

df_mysql = spark.read.format("jdbc") \
                    .option("url", f"jdbc:mysql://dockercompose-mysql-1:3306/dataops") \
                    .option("driver", "com.mysql.cj.jdbc.Driver") \
                    .option("query", sql) \
                    .option("user", "hyunsoo") \
                    .option("password", "910506") \
                    .load()

df_mysql.show(3)

+-----+---------+-----------+--------------------+------------------+--------------------+---+-----------+--------------------+
|index|birthdate|blood_group|                 job|              name|           residence|sex|        ssn|                uuid|
+-----+---------+-----------+--------------------+------------------+--------------------+---+-----------+--------------------+
|    0| 19810916|        AB+|Psychologist, pri...|      James Murray|PSC 1544, Box 316...|  M|741-53-6179|Cb7TYKQGDVZrB84Fo...|
|    1| 19580109|        AB-|Child psychothera...|     Sydney Weaver|96697 Marissa Byp...|  F|576-98-7289|4huCF5xaAGPLHyEpN...|
|    2| 19880909|         O+|Production assist...|Dr. Valerie Wagner|4981 Smith Prairi...|  F|196-87-0323|8AdYZ52fqepNjcizR...|
+-----+---------+-----------+--------------------+------------------+--------------------+---+-----------+--------------------+
only showing top 3 rows



#### 2. write functions

In [7]:
def write_file(df, fmt, path):
    df.write.format(fmt).mode("append").option("header", "true").save(path)
    
def write_mysql(df, db_name, table_name):
    df.write \
    .mode("append") \
    .format("jdbc") \
    .option("driver", "com.mysql.cj.jdbc.Driver") \
    .option("url", f"jdbc:mysql://dockercompose-mysql-1:3306/{db_name}") \
    .option("dbtable", table_name) \
    .option("user", "hyunsoo") \
    .option("password", "910506") \
    .save()    
    
def write_postgres(df, db_name, table_name):
    df.write \
    .mode("append") \
    .format("jdbc") \
    .option("driver", "org.postgresql.Driver") \
    .option("url", f"jdbc:postgresql://dockercompose-postgres-1:5432/{db_name}") \
    .option("dbtable", table_name) \
    .option("user", "yein") \
    .option("password", "941021") \
    .save()    

In [8]:
topic_name = "fake"

kafka_df = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "kafka1:19091,kafka2:19092,kafka3:19093") \
    .option("subscribe", topic_name) \
    .option("startingOffsets", "latest") \
    .load()

In [9]:
schema = T.StructType([
    T.StructField("birthdate", T.StringType()),
    T.StructField("blood_group", T.StringType()),
    T.StructField("job", T.StringType()),
    T.StructField("name", T.StringType()),
    T.StructField("residence", T.StringType()),
    T.StructField("sex", T.StringType()),
    T.StructField("ssn", T.StringType()),
    T.StructField("uuid", T.StringType()),
    T.StructField("timestamp", T.TimestampType()),
    ])

In [10]:
value_df = kafka_df.select(F.from_json(F.col("value").cast("string"), schema).alias("value"))

processed_df = value_df.selectExpr(
    "value.birthdate", 
    "value.blood_group", 
    "value.job",
    "value.name",
    "value.residence",
    "value.sex",
    "value.ssn",
    "value.uuid",
    "value.timestamp"
)

#### write to multiple databases

In [11]:
def data_output(df, batch_id):
    print(f"BATCH ID : {batch_id}")
    
    # write to MYSQL
    write_mysql(df, "dataops", "mysql_demo")
    
    # write to postgres
    write_postgres(df, "postgres", "postgres_demo")
    df.show()

In [12]:
df_batch = processed_df.writeStream \
    .foreachBatch(data_output) \
    .option("checkpointLocation", "checkpoint_dir/06_write_to_multiple_databases") \
    .trigger(processingTime="5 seconds") \
    .start()

df_batch.awaitTermination()

BATCH ID : 0
+---------+-----------+---+----+---------+---+---+----+---------+
|birthdate|blood_group|job|name|residence|sex|ssn|uuid|timestamp|
+---------+-----------+---+----+---------+---+---+----+---------+
+---------+-----------+---+----+---------+---+---+----+---------+

BATCH ID : 1
+---------+-----------+--------------------+----------------+--------------------+---+-----------+--------------------+-------------------+
|birthdate|blood_group|                 job|            name|           residence|sex|        ssn|                uuid|          timestamp|
+---------+-----------+--------------------+----------------+--------------------+---+-----------+--------------------+-------------------+
| 19100307|         O-|Horticulturist, c...|Jeffrey Williams|67312 Moss Ramp A...|  M|320-73-8383|j3GvuZ6gbGQZQoRQb...|2024-05-15 14:37:47|
| 19430719|         B-|Politician's assi...|Rebecca Campbell|4304 Nancy Field ...|  F|595-93-9767|jmEGgka46tbGNbvcL...|2024-05-15 14:37:49|
| 1986040

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/spark/python/lib/py4j-0.10.9.5-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/spark/python/lib/py4j-0.10.9.5-src.zip/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/local/lib/python3.7/socket.py", line 589, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 